In [1]:
import pandas as pd
from pymongo import MongoClient
import json
import datetime
import pprint
from bson.son import SON
    
def get_db(host,port,dbName):
    client = MongoClient(host,port)
    db = client[dbName]
    return db

def get_all_colls(db):
    return [i for i in db.collection_names()]

def get_specific_items(db, coll_name, time):
    items = db[coll_name].find({"datetime": {'$gte': time}},allow_partial_results=False).limit(5) # 注：原始db的datetime为ISO格式需要转换才能比较
    return items

def get_day_night_items(db, coll_name, date):
    items = db[coll_name].find({"date": {'$eq': date}},allow_partial_results=False) # 注：原始db的datetime为ISO格式需要转换才能比较
    return items

def get_daytime_items(db, coll_name, date, time):
    items = db[coll_name].find({"$and":[{"date":{"$eq": date}}, {"time":{"$lte": time}}]},allow_partial_results=False)
    return items

def coll_sum_by_date(db, coll_name):
    pipeline = [
        {"$unwind": "$date"},
        {"$group": {"_id": "$date", "count": {"$sum": 1}}},
        {"$sort": SON([("date", -1), ("_id", -1)])}
    ]
    sum = db[coll_name].aggregate(pipeline)
    return sum
    
db = get_db("localhost", 27017, 'VnTrader_Tick_Db')

colls = get_all_colls(db)
print colls

[u'm1901', u'rb1901', u'SR.HOT', u'p1901', u'SR901', u'm.HOT', u'rb.HOT', u'l1901', u'l.HOT', u'p.HOT']


In [2]:
coll_sum_by_date = coll_sum_by_date(db, 'rb1901')
pprint.pprint(list(coll_sum_by_date))

[{u'_id': u'20181026', u'count': 41386},
 {u'_id': u'20181025', u'count': 41402},
 {u'_id': u'20181024', u'count': 41396},
 {u'_id': u'20181023', u'count': 41393},
 {u'_id': u'20181022', u'count': 41401},
 {u'_id': u'20181019', u'count': 41403},
 {u'_id': u'20181018', u'count': 41400},
 {u'_id': u'20181017', u'count': 41403},
 {u'_id': u'20181016', u'count': 41396},
 {u'_id': u'20181015', u'count': 41404},
 {u'_id': u'20181012', u'count': 41394},
 {u'_id': u'20181011', u'count': 39261},
 {u'_id': u'20181010', u'count': 41402},
 {u'_id': u'20181009', u'count': 41351},
 {u'_id': u'20181008', u'count': 41395},
 {u'_id': u'20180928', u'count': 26996},
 {u'_id': u'20180927', u'count': 33664},
 {u'_id': u'20180926', u'count': 41386},
 {u'_id': u'20180925', u'count': 28734},
 {u'_id': u'20180921', u'count': 25222},
 {u'_id': u'20180920', u'count': 35621},
 {u'_id': u'20180919', u'count': 35542},
 {u'_id': u'20180918', u'count': 36513},
 {u'_id': u'20180917', u'count': 36346},
 {u'_id': u'2018

In [3]:
coll_name = 'rb1901'
date = '20181024'
time = '15:00:00.0'

cur = get_daytime_items(db, coll_name, date, time)

# print "daily tick data count is %s" % cur.count()
# print "cur type is %s"  % str(type(cur))
# print "cur[0] type is %s"  % str(type(cur[0]))
# print "cur[0:3] type is %s"  % str(type(cur[0:3]))
# print list(cur)

df = pd.DataFrame(list(cur))

cols=[x for i,x in enumerate(df.columns) if df.iat[0,i]==0 or df.iat[0,i] is None]
print cols
df = df.drop(cols,axis=1) # 删除值为0或none的列
df = df.set_index('datetime')
df.tail()

[u'askPrice2', u'askPrice3', u'askPrice4', u'askPrice5', u'askVolume2', u'askVolume3', u'askVolume4', u'askVolume5', u'bidPrice2', u'bidPrice3', u'bidPrice4', u'bidPrice5', u'bidVolume2', u'bidVolume3', u'bidVolume4', u'bidVolume5', u'lastVolume', u'rawData']


,_id,askPrice1,askVolume1,bidPrice1,bidVolume1,date,exchange,gatewayName,highPrice,lastPrice,lowPrice,lowerLimit,openInterest,openPrice,preClosePrice,symbol,time,upperLimit,volume,vtSymbol
datetime,,,,,,,,,,,,,,,,,,,,
2018-10-24 14:59:58.000,5bd0186e659aa0549523f760,4191.0,274,4190.0,1,20181024,SHFE,CTP,4193.0,4191.0,4112.0,3863.0,2903846.0,4128.0,4142.0,rb1901,14:59:58.0,4444.0,3393356,rb1901
2018-10-24 14:59:58.500,5bd0186e659aa0549523f765,4191.0,273,4190.0,1,20181024,SHFE,CTP,4193.0,4190.0,4112.0,3863.0,2903814.0,4128.0,4142.0,rb1901,14:59:58.5,4444.0,3393412,rb1901
2018-10-24 14:59:59.000,5bd0186f659aa0549523f76a,4190.0,9,4189.0,168,20181024,SHFE,CTP,4193.0,4190.0,4112.0,3863.0,2903614.0,4128.0,4142.0,rb1901,14:59:59.0,4444.0,3393640,rb1901
2018-10-24 14:59:59.500,5bd0186f659aa0549523f770,4190.0,28,4189.0,164,20181024,SHFE,CTP,4193.0,4190.0,4112.0,3863.0,2903560.0,4128.0,4142.0,rb1901,14:59:59.5,4444.0,3393712,rb1901
2018-10-24 15:00:00.000,5bd01870659aa0549523f777,4190.0,10,4189.0,155,20181024,SHFE,CTP,4193.0,4190.0,4112.0,3863.0,2903524.0,4128.0,4142.0,rb1901,15:00:00.0,4444.0,3393780,rb1901


In [26]:
import numpy as np
from __future__ import division

df['ret'] = df['lastPrice']/df['lastPrice'].shift(1) - 1
df['log_ret'] = np.log(df['lastPrice']/df['lastPrice'].shift(1))
df['vol_change'] = df['volume'] - df['volume'].shift(1)
df.fillna(value={'vol_change':0},inplace=True)

# def vwap(lastPrice, vol_change, volume):
#     if vol_change is NaN:
#         vwap = lastPrice
#     else:
#         vwap = vwap * 
        
#  if not self.lastTick:  # 根据第一个tick计算出的VWAP
#             self.VWAP = tick.lastPrice
#             self.volumeChange = tick.volume
#         elif self.lastTick:
#             self.volumeChange = tick.volume - self.lastTick.volume  # 当前tick的成交量=来的tick - 上一个tick（上一个tick来时已缓存）
#             self.VWAP = (self.VWAP * self.lastTick.volume + self.volumeChange * tick.lastPrice) / tick.volume

# a=(df.lastPrice * df.vol_change).sum()
# b=df.vol_change.sum()
# c=a/b
# c

# # df.info()
# def vwap(df):
#     df = df[['lastPrice', 'vol_change']]
#     denominator = (df.lastPrice * df.vol_change).sum()
#     numerator = df.vol_change.sum()
#     if denominator == 0:
#         return np.nan
#     return denominator/numerator

# # df.resample('10T', label='right').apply(vwap)
# df.apply(vwap)
# df.apply(lambda x: np.average(x['lastPrice'] ,weights = x['vol_change']) if sum(x['vol_change']) != 0 else np.average(x['lastPrice']),axis=1)
# sz50['hv60']=sz50['log_ret'].rolling(window=60,center=False).std() * np.sqrt(250)
# df.head()
# df[['lastPrice','log_ret']].plot(subplots=True,color='blue',figsize=(20,16))
df['askVol_bidVol_ratio'] = df['askVolume1'] / df['bidVolume1']
df.head()

,_id,askPrice1,askVolume1,bidPrice1,bidVolume1,date,exchange,gatewayName,highPrice,lastPrice,...,preClosePrice,symbol,time,upperLimit,volume,vtSymbol,ret,log_ret,vol_change,askVol_bidVol_ratio
datetime,,,,,,,,,,,,,,,,,,,,,
2018-10-24 09:00:00.500,5bcfc410659aa05495226161,4146.0,682,4145.0,55,20181024,SHFE,CTP,4154.0,4145.0,...,4142.0,rb1901,09:00:00.5,4444.0,1303606,rb1901,NaN,NaN,0.0,12.400000
2018-10-24 09:00:01.000,5bcfc411659aa05495226166,4146.0,599,4143.0,194,20181024,SHFE,CTP,4154.0,4143.0,...,4142.0,rb1901,09:00:01.0,4444.0,1304088,rb1901,-0.000483,-0.000483,482.0,3.087629
2018-10-24 09:00:01.500,5bcfc411659aa0549522616b,4145.0,53,4144.0,1,20181024,SHFE,CTP,4154.0,4145.0,...,4142.0,rb1901,09:00:01.5,4444.0,1304774,rb1901,0.000483,0.000483,686.0,53.000000
2018-10-24 09:00:02.000,5bcfc412659aa05495226170,4146.0,573,4145.0,36,20181024,SHFE,CTP,4154.0,4145.0,...,4142.0,rb1901,09:00:02.0,4444.0,1305462,rb1901,0.000000,0.000000,688.0,15.916667
2018-10-24 09:00:02.500,5bcfc412659aa05495226175,4145.0,2,4143.0,60,20181024,SHFE,CTP,4154.0,4146.0,...,4142.0,rb1901,09:00:02.5,4444.0,1305730,rb1901,0.000241,0.000241,268.0,0.033333


In [30]:
from sklearn import preprocessing

factor_scaled = preprocessing.scale(df['askVol_bidVol_ratio'])

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

nsample = len(df)
print nsample

x = factor_scaled       # 生成自变量数组x，其值在1-10等差排列
X = sm.add_constant(x)              # sm.add_constant是在向量左侧加上一列1

beta = np.array([1,nsample])             # 设置beta_0,beta_1真值为0,1

mu = np.random.normal(size=nsample) # 生成误差项mu,其值取自标准正态分布
y = np.dot(X,beta)+mu 

model = sm.OLS(y,X)    
results = model.fit()   # 模型求解
print results.params   # 获取模型参数估计值

26997
[  9.97537876e-01   2.69970012e+04]


In [31]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.958e+13
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        03:40:38   Log-Likelihood:                -38372.
No. Observations:               26997   AIC:                         7.675e+04
Df Residuals:                   26995   BIC:                         7.676e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.9975      0.006    163.506      0.000         0.986     1.009
x1            2.7e+04      0.006   4.43e+06      0.000       2.7e+04   2.7e+04
==============================================================================
Omnibus:                        2.818   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.244   Jarque-Bera (JB):                2.858
Skew:                           0.009   Prob(JB):                        0.240
Kurtosis:                       3.047   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
y_fitted = results.fittedvalues            # 调用fittedvalues得到拟合的y

plt.figure(figsize = (12,4))
plt.axis((0, 2, 0, 25))                    # 设置坐标轴区间
plt.grid(True)
plt.plot(x, y, 'o', label='data')          # 绘出原始数据
plt.plot(x, y_fitted, 'r--.',label='OLS')  # 绘出拟合数据
plt.legend(loc='best')                     # 添加图注释
plt.show()